# HYBRID RECOMMENDER SYSTEM
## İş Problemi
ID'siverilenkullanıcı için item-based ve user-based recommender yöntemlerini kullanarak 10film önerisi yapınız.
## Veri Seti Hikayesi
Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan
derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016
tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur
### movie.csv
#### movieId: Eşsiz film numarası. 
#### title: Film adı
#### genres: Tür
### rating.csv
#### userid: Eşsiz kullanıcı numarası. (UniqueID)
#### movieId: Eşsiz film numarası. (UniqueID)
#### rating: Kullanıcı tarafından filme verilen puan
#### timestamp: Değerlendirme tarihi

In [112]:
###########################
#Veriyi Hazırlama
###########################
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

rating=pd.read_csv("../datasets/rating.csv")
movie=pd.read_csv("../datasets/movie.csv")
df=rating.merge(movie,how="left",on="movieId")
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [113]:
#timestamp değişkenini kullanmyacağımız için dorplayalım
df.drop(["timestamp"],axis=1,inplace=True)
df.head()

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [114]:
# Toplam oy kullanılma sayısı 1000'nin altında olan filmleri veri setinden çıkaralım
total_rating=df.groupby("movieId").agg({"rating":"count"})
rare_movies=total_rating[total_rating["rating"]<1000].index
df=df[~df["movieId"].isin(rare_movies)]
df.shape

(17765516, 5)

In [115]:
# index'te userID'lerin sutunlarda film isimlerinin ve değer olarak ratinglerin bulunduğu dataframe oluşturalım
user_movie_df=df.pivot_table(index=["userId"],columns=["movieId"],values="rating")
user_movie_df.head()

movieId  1       2       3       4       5       6       7       8       9       10      11      12      13      14      15      16      17      18      19      20      21      22      23      24      25      26      27      28      29      30      31      32      34      35      36      39      41      42      43      44      45      46      47      48      50      52      57      58      60      61      62      63      64      65      66      68      69      70      71      72      73      74      76      78      79      81      82      85      86      88      89      92      93      94      95      97      100     101     102     103     104     105     107     110     111     112     113     117     118     122     123     125     126     132     135     140     141     144     145     147     149     150     151     153     154     155     156     157     158     159     160     161     162     163     164     165     168     169     170     171     172     173     174     175     176     177     179     180     181     185     186     188     191     193     194     195     196     198     199     203     204     205     206     207     208     213     215     216     217     218     222     223     224     225     227     229     230     231     232     233     234     235     236     237     239     242     246     247     248     249     252     253     255     256     257     258     259     260     261     262     265     266     267     270     271     272     273     274     275     276     277     278     280     281     282     288     289     290     292     293     296     299     300     302     303     305     306     307     308     312     313     314     315     316     317     318     319     322     325     326     327     328     329     330     332     333     334     337     338     339     340     342     344     345     346     347     348     349     350     351     352     353     354     355     356     357     358     360     361     362     364     365     366     367     368     369     370     371     372     373     374     376     377     378     379     380     381     382     383     393     405     407     410     412     413     415     416     417     419     420     421     422     423     426     427     428     429     431     432     433     434     435     436     437     438     440     441     442     444     445     446     448     450     452     454     455     457     458     459     464     466     468     469     471     473     474     475     477     479     480     481     482     485     489     490     491     492     493     494     497     500     501     502     504     505     506     507     508     509     511     512     514     515     516     517     518     519     520     521     522     524     527     529     531     532     533     534     535     537     538     539     540     541     542     543     544     546     548     549     550     551     552     553     555     556     558     562     569     574     575     581     585     586     587     588     589     590     592     593     594     595     596     597     599     605     606     608     609     610     611     612     613     616     627     628     631     635     637     639     640     647     648     650     653     656     661     662     663     665     667     671     673     674     678     680     688     691     694     697     700     704     707     708     709     710     711     714     719     720     724     725     728     733     735     736     737     741     742     743     745     747     748     750     761     762     765     766     778     780     781     782     783     784     785     786     788     798     799     800     801     802     804     805     809     810     818     828     829     830     832     833     836     837     838     839     842     848     849     851     852     858     861     866     879     880     881     886     891     892    

In [116]:
#Yaptığımız veri hazırlama işlemlerini fonksiyonlaştıralım
def data_preparation():
    import pandas as pd
    rating=pd.read_csv("../datasets/rating.csv")
    movie=pd.read_csv("../datasets/movie.csv")
    df=rating.merge(movie,how="left",on="movieId")
    #timestamp değişkenini kullanmyacağımız için dorplayalım
    df.drop(["timestamp"],axis=1,inplace=True)
    # Toplam oy kullanılma sayısı 1000'nin altında olan filmleri veri setinden çıkaralım
    total_rating=df.groupby("movieId").agg({"rating":"count"})
    rare_movies=total_rating[total_rating["rating"]<1000].index
    df=df[~df["movieId"].isin(rare_movies)]
    # index'te userID'lerin sutunlarda film isimlerinin ve değer olarak ratinglerin bulunduğu dataframe oluşturalım
    user_movie_df=df.pivot_table(index=["userId"],columns=["movieId"],values="rating")
    return user_movie_df

## User-Based Recommendation

In [117]:
####################################################################
#Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
####################################################################
#Rastgele bir kullanıcı seçip onun için user_movie_df oluşturalım
random_user=119048
random_user_df=user_movie_df[user_movie_df.index==random_user]
random_user_df


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,113,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,206,207,208,213,215,216,217,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,255,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,325,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,369,370,371,372,373,374,376,377,378,379,380,381,382,383,393,405,407,410,412,413,415,416,417,419,420,421,422,423,426,427,428,429,431,432,433,434,435,436,437,438,440,441,442,444,445,446,448,450,452,454,455,457,458,459,464,466,468,469,471,473,474,475,477,479,480,481,482,485,489,490,491,492,493,494,497,500,501,502,504,505,506,507,508,509,511,512,514,515,516,517,518,519,520,521,522,524,527,529,531,532,533,534,535,537,538,539,540,541,542,543,544,546,548,549,550,551,552,553,555,556,558,562,569,574,575,581,585,586,587,588,589,590,592,593,594,595,596,597,599,605,606,608,609,610,611,612,613,616,627,628,631,635,637,639,640,647,648,650,653,656,661,662,663,665,667,671,673,674,678,680,688,691,694,697,700,704,707,708,709,710,711,714,719,720,724,725,728,733,735,736,737,741,742,743,745,747,748,750,761,762,765,766,778,780,781,782,783,784,785,786,788,798,799,800,801,802,804,805,809,810,818,828,829,830,832,833,836,837,838,839,842,848,849,851,852,858,861,866,879,880,881,886,891,892,898,899,900,901,902,903,904,905,906,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,926,928,930,931,932,933,934,936,938,940,942,943,944,945,946,947,948,949,950,951,952,953,954,955,965,968,969,971,986,991,994,996,999,1003,1004,1005,1006,1007,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1041,1042,1043,1046,1047,1049,1050,1051,1057,1059,1060,1061,1064,1066,1073,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1099,1100,1101,1103,1104,1111,1113,1120,1124,1125,1126,1127,1128,1129,1130,1131,1132,1135,1136,1147,1148,1150,1161,1171,1172,1173,1175,1176,1177,1178,1179,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1196,1197,1198,1199,1200,1201,1202,1203,1204,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1230,1231,1232,1233,1234,1235,1237,1238,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1292,1293,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1320,1321,1327,1333,1334,1339,1340,1342,1343,1344,1345,1346,1347,1348,1350,1353,1354,1356,1357,1358,1359,1361,1363,1365,1366,1367,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1385,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1399,1401,1405,1407,1408,1409,1411,1414,1416,1419,1422,1425,1429,1431,1432,1438,1441,1445,1446,1449,1453,1457,1459,1461,1464,1466,1468,1474,1476,1479,1480,1483,1484,1485,1487,1488,1498,1499,1500,1503,1513,1515,1517,1518,1527,1537,1541,1542,1544,1552,1554,1556,1562,1566,1569,1573,1580,1584,1586,1587,1588,1589,1590,1591,1592,1593,1594,1597,1603,1608,1610,1611,1614,1615,1616,1617,1619,1620,1623,1625,1627,1633,1635,1639,1641,1643,1644,1645,1648,1649,1653,1658,1660,1663,1665,1667,1672,1673,1674,1676,1678,1680,

In [118]:
#Seçilen kullancıların oy kullandığı filmleri bir listede toplayalım
watched_movies=[]
for col in random_user_df.columns:
    if(random_user_df[col].values[0]>=0):
        watched_movies.append(col)
watched_movies[0:10]


[1, 2, 3, 5, 6, 7, 10, 12, 13, 14]

In [119]:
#Random userın izlediği filmleri user_movie_df'in sütunlarından seçerek  movies_watched_df oluşturalım
movies_watched_df=user_movie_df[watched_movies]
movies_watched_df

movieId  1       2       3       5       6       7       10      12      13      14      15      17      18      19      21      22      23      24      25      31      32      34      36      39      43      44      45      47      48      57      60      64      65      69      72      85      88      94      102     104     105     111     123     125     126     135     141     144     150     151     153     154     158     160     165     169     170     171     174     175     181     185     186     196     198     199     203     206     208     218     223     231     234     235     236     237     239     247     248     249     252     253     256     260     261     266     267     271     272     274     275     276     278     281     282     288     292     293     296     299     300     302     306     307     308     312     316     317     318     319     327     329     332     333     334     337     338     339     342     344     345     348     350     355     356     357     361     364     366     367     368     370     372     374     376     377     379     380     381     382     393     410     412     413     419     423     429     432     435     436     440     441     442     446     448     455     457     466     471     473     475     477     479     480     492     493     494     497     500     501     505     506     507     508     509     514     515     516     520     524     527     529     531     532     533     534     535     538     540     541     542     543     546     550     551     552     558     562     575     585     586     587     588     589     592     593     594     595     596     597     599     608     611     612     616     627     628     637     647     648     653     656     661     663     671     673     688     708     711     719     724     736     743     748     750     761     765     766     778     780     783     788     798     799     800     801     802     818     829     830     833     836     838     851     852     858     866     879     898     899     900     901     902     903     904     905     906     908     909     910     911     912     913     914     915     916     918     919     920     921     922     923     924     926     928     930     931     932     933     936     938     940     942     943     945     946     947     948     949     950     951     952     953     954     955     965     968     969     971     1010    1013    1015    1016    1017    1020    1021    1022    1023    1024    1025    1028    1029    1030    1031    1032    1033    1035    1036    1041    1042    1046    1057    1059    1061    1064    1066    1073    1077    1079    1081    1084    1086    1089    1090    1091    1094    1095    1096    1097    1103    1104    1111    1126    1127    1135    1136    1148    1171    1177    1178    1179    1183    1185    1186    1188    1189    1192    1193    1196    1198    1199    1200    1203    1204    1206    1208    1210    1211    1212    1213    1214    1217    1219    1221    1223    1226    1228    1230    1234    1235    1237    1240    1244    1245    1246    1247    1248    1250    1251    1252    1253    1254    1256    1258    1259    1260    1263    1265    1266    1267    1269    1270    1274    1276    1278    1280    1281    1282    1283    1284    1285    1287    1288    1291    1296    1300    1301    1304    1305    1307    1320    1321    1333    1334    1339    1340    1343    1344    1345    1347    1348    1354    1356    1361    1366    1367    1372    1373    1376    1377    1380    1387    1391    1392    1393    1394    1396    1399    1405    1407    1408    1416    1419    1425    1431    1438    1441    1449    1453    1457    1461    1464    1466    1480    1484    1485    1499    1513    1515    1517    1527    1544    1556    1562    1566    1569    1580    1584    1588    1590    1591    1594    1603    1608    1611    1614    1617    1635    1639   

In [120]:
#Her bir kullancının seçili user'in izlediği filmlerin kaçını izlediğini bilgisini taşıyan 
#user_movie_count adında yeni bir dataframe oluşturunuz
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count=user_movie_count.reset_index()
user_movie_count.columns=["userId","NumberOfWatchedMovie"]
user_movie_count

,userId,NumberOfWatchedMovie
0,1,110
1,2,30
2,3,132
3,4,21
4,5,54
...,...,...
138488,138489,29
138489,138490,79
138490,138491,15
138491,138492,59


In [121]:
#Random userın izlediği filmlerin %60 ve fazlasını izlyen userların userId'sini bir listede tutalım
users_same_movies=user_movie_count[user_movie_count["NumberOfWatchedMovie"]>=len(watched_movies)*0.6]["userId"]
users_same_movies=list(users_same_movies)
len(users_same_movies)


87

In [122]:
#User_same_movies listesi içerisindeki seçili user ile benzerlik gösteren kullanıcıların 
#id’lerinin bulunacağı şekilde movies_watched_df dataframe’ini filtreleyelim

final_df=movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
final_df


movieId  1       2       3       5       6       7       10      12      13      14      15      17      18      19      21      22      23      24      25      31      32      34      36      39      43      44      45      47      48      57      60      64      65      69      72      85      88      94      102     104     105     111     123     125     126     135     141     144     150     151     153     154     158     160     165     169     170     171     174     175     181     185     186     196     198     199     203     206     208     218     223     231     234     235     236     237     239     247     248     249     252     253     256     260     261     266     267     271     272     274     275     276     278     281     282     288     292     293     296     299     300     302     306     307     308     312     316     317     318     319     327     329     332     333     334     337     338     339     342     344     345     348     350     355     356     357     361     364     366     367     368     370     372     374     376     377     379     380     381     382     393     410     412     413     419     423     429     432     435     436     440     441     442     446     448     455     457     466     471     473     475     477     479     480     492     493     494     497     500     501     505     506     507     508     509     514     515     516     520     524     527     529     531     532     533     534     535     538     540     541     542     543     546     550     551     552     558     562     575     585     586     587     588     589     592     593     594     595     596     597     599     608     611     612     616     627     628     637     647     648     653     656     661     663     671     673     688     708     711     719     724     736     743     748     750     761     765     766     778     780     783     788     798     799     800     801     802     818     829     830     833     836     838     851     852     858     866     879     898     899     900     901     902     903     904     905     906     908     909     910     911     912     913     914     915     916     918     919     920     921     922     923     924     926     928     930     931     932     933     936     938     940     942     943     945     946     947     948     949     950     951     952     953     954     955     965     968     969     971     1010    1013    1015    1016    1017    1020    1021    1022    1023    1024    1025    1028    1029    1030    1031    1032    1033    1035    1036    1041    1042    1046    1057    1059    1061    1064    1066    1073    1077    1079    1081    1084    1086    1089    1090    1091    1094    1095    1096    1097    1103    1104    1111    1126    1127    1135    1136    1148    1171    1177    1178    1179    1183    1185    1186    1188    1189    1192    1193    1196    1198    1199    1200    1203    1204    1206    1208    1210    1211    1212    1213    1214    1217    1219    1221    1223    1226    1228    1230    1234    1235    1237    1240    1244    1245    1246    1247    1248    1250    1251    1252    1253    1254    1256    1258    1259    1260    1263    1265    1266    1267    1269    1270    1274    1276    1278    1280    1281    1282    1283    1284    1285    1287    1288    1291    1296    1300    1301    1304    1305    1307    1320    1321    1333    1334    1339    1340    1343    1344    1345    1347    1348    1354    1356    1361    1366    1367    1372    1373    1376    1377    1380    1387    1391    1392    1393    1394    1396    1399    1405    1407    1408    1416    1419    1425    1431    1438    1441    1449    1453    1457    1461    1464    1466    1480    1484    1485    1499    1513    1515    1517    1527    1544    1556    1562    1566    1569    1580    1584    1588    1590    1591    1594    1603    1608    1611    1614    1617    1635    1639   

In [123]:
#Kullanıcıların birbiriyle olan korelasyonun(benzerliği) bulunacağı yeni bir corr_df oluşturalım
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

corr_df = pd.DataFrame(corr_df, columns=["corr"])

corr_df.index.names = ['user_id_1', 'user_id_2']

corr_df = corr_df.reset_index()
corr_df

,user_id_1,user_id_2,corr
0,116317,7201,-0.093232
1,135425,116317,-0.078616
2,107640,52009,-0.053148
3,116317,114406,-0.050055
4,62812,14705,-0.048096
...,...,...,...
3737,8405,75299,0.743687
3738,89307,135090,0.762848
3739,20132,82418,0.770120
3740,119048,54107,0.952035


In [124]:
#Seçili kullanıcı ile yüksek korelasyona sahip (0.60’ın üzerinde olan) kullanıcıları filtreleyerek 
#top_users adında yeni bir dataframe oluşturalım.
top_users1=corr_df[(corr_df["user_id_1"]==random_user) & (corr_df["corr"]>0.60)][["user_id_2","corr"]].reset_index(drop=True).sort_values("corr",
                                                                                                                   ascending=False)
top_users2=corr_df[(corr_df["user_id_2"]==random_user) & (corr_df["corr"]>0.60)][["user_id_1","corr"]].reset_index(drop=True).sort_values("corr",
                                                                                                                   ascending=False)
top_users2.columns=["user_id_2","corr"]
top_users=pd.concat([top_users1,top_users2])
top_users=top_users.sort_values("corr",ascending=False).reset_index(drop=True)
top_users



,user_id_2,corr
0,54107,0.952035
1,8405,0.743511
2,100134,0.688462
3,14705,0.682736
4,75299,0.652980
5,18138,0.651647
6,25101,0.649661
7,114406,0.621045
8,35128,0.614409
9,75810,0.613600


In [125]:
#top_users dataframe'ine rating dataframenini merge edelim
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users=top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users

,userId,corr,movieId,rating
0,54107,0.952035,1,5.0
1,54107,0.952035,2,2.0
2,54107,0.952035,3,2.5
3,54107,0.952035,5,3.0
4,54107,0.952035,6,4.5
...,...,...,...,...
37192,88820,0.604376,125950,3.5
37193,88820,0.604376,129285,3.5
37194,88820,0.604376,130204,3.0
37195,88820,0.604376,130328,3.0


In [126]:
# Her bir kullanıcının corr ve rating değerlerinin çarpımından oluşan weighted_rating adında yeni bir değişken oluşturalım
top_users["weighted_rating"]=top_users["rating"]*top_users["corr"]
top_users.head()

,userId,corr,movieId,rating,weighted_rating
0,54107,0.952035,1,5.0,4.760175
1,54107,0.952035,2,2.0,1.904070
2,54107,0.952035,3,2.5,2.380087
3,54107,0.952035,5,3.0,2.856105
4,54107,0.952035,6,4.5,4.284157


In [127]:
# Film idsi ve o filme ait weighted_ratingslerin ortalaması olan bir recommendation_df oluşturalım
recommendation_df=top_users.groupby("movieId").agg({"weighted_rating":"mean"})
recommendation_df.reset_index(inplace=True)
recommendation_df

,movieId,weighted_rating
0,1,3.162308
1,2,1.316986
2,3,1.575480
3,4,1.293395
4,5,1.488276
...,...,...
11153,126595,3.332122
11154,129285,2.115317
11155,130204,1.813129
11156,130328,1.813129


In [128]:
# recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan filmleri seçip  weighted rating’e göre sıralayalım
recommendation_df=recommendation_df[recommendation_df["weighted_rating"]>=3.5].sort_values("weighted_rating",ascending=False)
recommendation_df.head()

,movieId,weighted_rating
7898,33912,4.760175
9920,74416,4.760175
11110,112183,4.760175
10863,97024,4.760175
11098,111251,4.760175


In [129]:
# movie veri setinden film isimlerini getiriniz ve tavsiye edilecek ilk 5 filmi seçelim
recommended_films=list(recommendation_df.head()["movieId"])
recommended_films

[33912, 74416, 112183, 97024, 111251]

## Item-Based Recommendation

In [130]:
# Random userın 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin alınız.
df=rating.merge(movie,how="left",on="movieId")
random_user_five_points=df[(df["userId"]==random_user)&(df["rating"]==5)]
most_recent_date=random_user_five_points["timestamp"].max()
most_recent_film=random_user_five_points[random_user_five_points["timestamp"]==most_recent_date]["movieId"].values[0]
most_recent_film

3075

In [131]:
#User based recommendation bölümünde oluşturulan user_movie_df dataframe’ini seçilen film id’sine göre filtreleyelim.
recent_film_df=user_movie_df[most_recent_film]
recent_film_df

userId
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
          ..
138489   NaN
138490   NaN
138491   NaN
138492   NaN
138493   NaN
Name: 3075, Length: 138493, dtype: float64

In [133]:
# Filtrelenen dataframe’i kullanarak seçili filmle diğer filmlerin korelasyonunu bulup sıralayalım.
similar_movies=user_movie_df.corrwith(recent_film_df).sort_values(ascending=False)
similar_movies.head()

movieId
3075    1.000000
2488    0.538234
7116    0.519867
1085    0.493318
7766    0.464658
dtype: float64

In [137]:
# Seçili film’in kendisi haricinde ilk 5 film’I öneri olarak verelim
item_based_recommended=list(similar_movies.index[1:6])
item_based_recommended

[2488, 7116, 1085, 7766, 8228]

## Result of Hybrid Recommendation

In [138]:
#Önerilen filmleri tek bir listede toplayalım
hybrid_recommended=item_based_recommended+recommended_films
hybrid_recommended

[2488, 7116, 1085, 7766, 8228, 33912, 74416, 112183, 97024, 111251]

In [144]:
# Girilen anahtar kelimeyi barındıran film isimlerini getiren bir fonksiyon yazalım
# ve random userın en son 5 puan verdiği filmi bulalım
def check_name(id):
    return movie[movie["movieId"]==id]["title"].values[0]
check_name(most_recent_film)

'Repulsion (1965)'

In [146]:
# Önerdiğimiz filmlerin de ismini bir listede tutalım
hybrid_recommended_name=[check_name(id) for id in hybrid_recommended]
hybrid_recommended_name

['Peeping Tom (1960)',
 'Diabolique (Les diaboliques) (1955)',
 'Old Man and the Sea, The (1958)',
 'Throne of Blood (Kumonosu jô) (1957)',
 'Maltese Falcon, The (a.k.a. Dangerous Female) (1931)',
 'Unmarried Woman, An (1978)',
 'Fish Tank (2009)',
 'Birdman (2014)',
 "Rust and Bone (De rouille et d'os) (2012)",
 'Immigrant, The (2013)']

In [148]:
#Girilen filmin türüne veren bir fonksiyon yazalım ve elimizdeki filmlerin türlerine bakalım
def check_genres(name):
    return movie[movie["title"]==name]["genres"].values[0]
print("Random userın en son 5 puan verdiği filmin türü: {}".format(check_genres(check_name(most_recent_film))))

Random userın en son 5 puan verdiği filmin türü: Drama|Horror


In [149]:
# Önerdiğimiz filmlerin türlerine bakalım
for film in hybrid_recommended_name:
    print(check_genres(film))


Drama|Horror|Thriller
Horror|Mystery|Thriller
Adventure|Drama
Action|Drama|Thriller|War
Mystery
Comedy|Drama|Romance
Drama
Comedy|Drama
Drama|Romance
Drama|Romance
